# Bernoulli Console With Random Orientation and Global Distributed Loads

In [1]:
L, d, t = 1000.0, 100.0, 5.0
Ex, nu = 210000.0, 0.25

In [2]:
from linkeddeepdict import LinkedDeepDict
from linkeddeepdict.tools import getallfromkwargs
from neumann.linalg import Vector, linspace
from neumann.array import repeat
from polymesh.space import StandardFrame, PointCloud, frames_of_lines
from polymesh.space.utils import index_of_closest_point, index_of_furthest_point
from polymesh.topo.tr import L2_to_L3
from sigmaepsilon.solid.fem.cells import B2, B3
from sigmaepsilon.solid import Structure, LineMesh, PointData, BeamSection
import numpy as np
from latexdocs.utils import floatformatter

seed = 0  # integer or None
seed = None

rs = np.random.RandomState()
if isinstance(seed, int):
    rs.seed(seed)

formatter = floatformatter(sig=4)
def f2s(x): return formatter.format(x)


# section
section = BeamSection('CHS', d=d, t=t, n=32)
section.calculate_section_properties()
section_props = section.section_properties
A, Ix, Iy, Iz = getallfromkwargs(['A', 'Ix', 'Iy', 'Iz'], **section_props)


# material
G = Ex / (2 * (1 + nu))
Hooke = np.array([
    [Ex*A, 0, 0, 0],
    [0, G*Ix, 0, 0],
    [0, 0, Ex*Iy, 0],
    [0, 0, 0, Ex*Iz]
])


def solve(n, angles, loads, celltype=B2):
    # space
    GlobalFrame = StandardFrame(dim=3)
    TargetFrame = GlobalFrame.rotate('Body', angles, 'XYZ', inplace=False)

    # mesh
    p0 = np.array([0., 0., 0.])
    p1 = np.array([L, 0., 0.])
    coords = linspace(p0, p1, n+1)
    points = PointCloud(coords, frame=TargetFrame)
    coords = points.show(GlobalFrame)
    topo = np.zeros((n, 2), dtype=int)
    topo[:, 0] = np.arange(n)
    topo[:, 1] = np.arange(n) + 1
    if celltype.NNODE == 3:
        coords, topo = L2_to_L3(coords, topo)
    i_first = index_of_closest_point(coords, np.array([0., 0., 0.]))
    i_last = index_of_furthest_point(coords, np.array([0., 0., 0.]))

    # essential boundary conditions
    penalty = 1e40  # penalty value
    fixity = np.zeros((coords.shape[0], 6)).astype(bool)
    fixity[i_first, :] = True
    fixity = fixity.astype(float) * penalty

    # natural boundary conditions
    loads = np.array(loads)
    loads[:3] = Vector(loads[:3], frame=GlobalFrame).show(TargetFrame)
    nodal_loads = np.zeros((coords.shape[0], 6))
    cell_loads = np.zeros((topo.shape[0], topo.shape[1], 6))
    cell_loads[:, :, :] = loads

    # pointdata
    pd = PointData(coords=coords, frame=GlobalFrame,
                   loads=nodal_loads, fixity=fixity)

    # celldata
    frames = repeat(TargetFrame.axes, topo.shape[0])
    cd = celltype(topo=topo, loads=cell_loads, frames=frames)

    # set up mesh and structure
    mesh = LineMesh(pd, cd, model=Hooke, frame=GlobalFrame)
    structure = Structure(mesh=mesh)

    structure.linsolve()

    dofsol = structure.nodal_dof_solution(store='dofsol')
    
    reactions = structure.reaction_forces()
    return reactions[i_first]


In [3]:
loads = [0, 0, 1.0, 0, 0, 0]
n = 2  # number of elements
cells = [B2, B3]

for cell in cells:
    for _ in range(10):
        angles = np.random.rand(3) * np.pi * 2
        r = solve(n, angles, loads, cell)
        print([f2s(v) for v in r[:3]])

['-9.767e-12', '3.212e-12', '-1000']
['4.791e-14', '-1.644e-12', '-1000']
['-2.145e-13', '2.588e-12', '-1000']
['-1.087e-13', '-1.397e-13', '-1000']
['-5.452e-12', '1.652e-13', '-1000']
['2.183e-13', '-3.575e-13', '-1000']
['-3.414e-12', '-1.882e-12', '-1000']
['-3.362e-13', '-5.572e-13', '-1000']
['7.562e-13', '3.147e-12', '-1000']
['8.846e-13', '3.644e-12', '-1000']
['8.457e-12', '2.583e-11', '-1000']
['1.961e-11', '8.616e-12', '-1000']
['4.218e-11', '3.71e-11', '-1000']
['1.289e-12', '6.729e-13', '-1000']
['-6.286e-11', '3.186e-11', '-1000']
['-1.07e-10', '3.852e-11', '-1000']
['3.735e-11', '-2.616e-11', '-1000']
['3.565e-12', '-6.143e-12', '-1000']
['-6.69e-12', '-3.787e-11', '-1000']
['9.801e-11', '-1.121e-11', '-1000']


In [4]:
def solve(n, angles, loads, celltype=B2):
    # space
    GlobalFrame = StandardFrame(dim=3)
    angles[1:] = 0.
    TargetFrame = GlobalFrame.rotate('Body', angles, 'XYZ', inplace=False)

    # mesh
    p0 = np.array([0., 0., 0.])
    p1 = np.array([L, 0., 0.])
    coords = linspace(p0, p1, n+1)
    topo = np.zeros((n, 2), dtype=int)
    topo[:, 0] = np.arange(n)
    topo[:, 1] = np.arange(n) + 1
    if celltype.NNODE == 3:
        coords, topo = L2_to_L3(coords, topo)
    i_first = index_of_closest_point(coords, np.array([0., 0., 0.]))
    i_last = index_of_furthest_point(coords, np.array([0., 0., 0.]))

    # essential boundary conditions
    penalty = 1e40  # penalty value
    fixity = np.zeros((coords.shape[0], 6)).astype(bool)
    fixity[i_first, :] = True
    fixity = fixity.astype(float) * penalty

    # natural boundary conditions
    loads = np.array(loads)
    loads[:3] = Vector(loads[:3], frame=GlobalFrame).show(TargetFrame)
    nodal_loads = np.zeros((coords.shape[0], 6))
    cell_loads = np.zeros((topo.shape[0], topo.shape[1], 6))
    cell_loads[:, :, :] = loads

    # pointdata
    pd = PointData(coords=coords, frame=GlobalFrame,
                   loads=nodal_loads, fixity=fixity)

    # celldata
    frames = repeat(TargetFrame.axes, topo.shape[0])
    cd = celltype(topo=topo, loads=cell_loads, frames=frames)

    # set up mesh and structure
    mesh = LineMesh(pd, cd, model=Hooke, frame=GlobalFrame)
    structure = Structure(mesh=mesh)

    structure.linsolve()

    dofsol = structure.nodal_dof_solution(store='dofsol')
    
    reactions = structure.reaction_forces()
    return reactions[i_first]

In [5]:
loads = [0, 0, 1.0, 0, 0, 0]
n = 2  # number of elements
cells = [B2, B3]

for cell in cells:
    for _ in range(10):
        angles = np.random.rand(3) * np.pi * 2
        r = solve(n, angles, loads, cell)
        print([f2s(v) for v in r[:3]])

['0', '2.842e-14', '-1000']
['0', '1.776e-15', '-1000']
['0', '1.137e-13', '-1000']
['0', '-1.421e-14', '-1000']
['0', '-1.776e-15', '-1000']
['0', '1.421e-14', '-1000']
['0', '1.421e-14', '-1000']
['0', '0', '-1000']
['0', '8.078e-28', '-1000']
['0', '0', '-1000']
['0', '2.796e-11', '-1000']
['0', '2.881e-11', '-1000']
['0', '-6.533e-11', '-1000']
['0', '-6.056e-12', '-1000']
['0', '5.03e-11', '-1000']
['0', '1.338e-11', '-1000']
['0', '4.265e-11', '-1000']
['0', '2.829e-11', '-1000']
['0', '2.667e-11', '-1000']
['0', '9.789e-12', '-1000']
